In [67]:
from pysat.formula import CNF
from pysat.solvers import *
import re
import pandas as pd
import os

CONFIG_DIR = "../linux-4.14.152"
DIMACS_FLA = 'examples/out.dimacs'
CONFIG = "../linux-4.14.152/.config"
CSV = 'examples/alloptions-x86.4.14.152.csv'
CONF_FLAG = ["randconfig"] # , "defconfig", "allnoconfig", "allyesconfig", "allmodconfig", "alldefconfig"]

In [81]:
variables = {}
with open(DIMACS_FLA, 'r') as f:
    dimacs_lines = f.readlines()
    for line in dimacs_lines:
        if line.startswith("c"):
            m = re.search('(c) (\d+) (\w+)', line)
            var_id = int(m.group(2))
            var_name = m.group(3)  
            variables[var_id] = var_name

def search_kconfig_type(option_name, df_options):
    ktype = df_options.query("option == " + '"' + option_name + '"')['type']
    return ktype.values[0]

# we are thinking the DIMACS id (number k) corresponding to the "module" variable of an option
def search_kmodule(option_name):
    for k, v in variables.items():
        if v == option_name + "_MODULE":
            return k

def get_dict(filename):
    """Dictionary from a linux config file"""
    res = dict()
    with open(CONFIG, 'r') as f:
        config_lines = f.readlines()
        for line in config_lines:
            if not line.startswith("#") and line != '\n':
                m = re.search('CONFIG_(\w+)=([\w"-/]+)', line)
                var_name = m.group(1)
                var_val = m.group(2)
                res[var_name] = var_val
    return res

def get_var(option):
    for k, v in variables.items():
        if v == option:
            return k
        
def tristate_repartition(config_dict):
    return {'yes' : list(config_dict.values()).count('y'),
            'mod' : list(config_dict.values()).count('m')}
        

In [89]:
def check_both(config_dict):
    fla = CNF(DIMACS_FLA)
    # we read once and for all
    df_options = pd.read_csv(CSV)
    assumptions = []
    nb_yes = 0
    nb_mod = 0
    with Solver(bootstrap_with=fla.clauses) as l:
        print("at the beginning, fla is", l.solve(assumptions=[]))
        print("=> ADDING FEATURES FROM THE .CONFIG FILE [BOOL&TRISTATE]")
        for k, v in variables.items():
            if v in config_dict:
                if (v.endswith("_MODULE") or "CHOICE_" in v):  
                    continue # ignore
                kconfig_type = search_kconfig_type(v, df_options)
                if (kconfig_type == 'TRISTATE'):
                    # we have to find the other "module" variable that emulates the "m" value 
                    kmodule = search_kmodule(v)
                    # YES
                    if config_dict[v] == 'y':
                        print("+ Adding {} ({})".format(v, k))
                        print("+ Adding {} ({})".format(v + "_MODULE", -kmodule))
                        assumptions.extend([k, -kmodule])
                        nb_yes += 1
                    # MODULE
                    elif config_dict[v] == 'm':
                        print("+ Adding {} ({})".format(v, -k))
                        print("+ Adding {} ({})".format(v + "_MODULE", kmodule))
                        assumptions.extend([-k, kmodule])
                        nb_mod += 1
                    # SAT Testing
                    if not l.solve(assumptions=assumptions):
                        print("== UNSAT")
                        core = list(map((lambda x : '~' + variables[abs(x)] if (x < 0) else variables[x]),
                                        l.get_core()))
                        print(core)
                        return {"sat" : False,
                                "last_clause" : v,
                                "in" : True,
                                "core" : core}
                elif (kconfig_type == 'BOOL'):
                    # we have to find the other "module" variable that emulates the "m" value 
                    kmodule = search_kmodule(v)
                    print("+ Adding {} ({})".format(v, k))
                    assumptions.extend([k])
                    # SAT Testing
                    if not l.solve(assumptions=assumptions):
                        print("== UNSAT")
                        core = list(map((lambda x : '~' + variables[abs(x)] if (x < 0) else variables[x]),
                                        l.get_core()))
                        print(core)
                        return {"sat" : False,
                                "last_clause" : v,
                                "in" : True,
                                "core" : core}
        print("=> ADDING FEATURES THAT ARE NOT IN THE .CONFIG FILE [BOOL&TRISTATE]")
        for f in df_options.option:
            # Feature in the CSV and in the DIMACS but not in the config
            if f not in config and f in variables.values():
                k = get_var(f)
                kconfig_type = search_kconfig_type(f, df_options)
                if (kconfig_type == 'TRISTATE'):
                    kmodule = search_kmodule(f)
                    # NO
                    print("+ Adding {} ({})".format(f, -k))
                    print("+ Adding {} ({})".format(f + "_MODULE", -kmodule))
                    assumptions.extend([-k, -kmodule])
                    # SAT Testing
                    if not l.solve(assumptions=assumptions):
                        print("== UNSAT")
                        core = list(map((lambda x : '~' + variables[abs(x)] if (x < 0) else variables[x]),
                                        l.get_core()))
                        print(core)
                        return {"sat" : False,
                            "last_clause" : f,
                            "in" : True,
                            "core" : core}
                elif (kconfig_type == 'BOOL'):
                    kmodule = search_kmodule(f)
                    # NO
                    print("+ Adding {} ({})".format(f, -k))
                    assumptions.extend([-k])
                    # SAT Testing
                    if not l.solve(assumptions=assumptions):
                        print("== UNSAT")
                        core = list(map((lambda x : '~' + variables[abs(x)] if (x < 0) else variables[x]),
                                        l.get_core()))
                        print(core)
                        return {"sat" : False,
                            "last_clause" : f,
                            "in" : True,
                            "core" : core}
        print("== SAT")
        return {"sat" : False,
                "last_clause" : None,
                "in" : True,
                "core" : None}

In [91]:
def check_tristate(config_dict):
    fla = CNF(DIMACS_FLA)
    # we read once and for all
    df_options = pd.read_csv(CSV)
    assumptions = []
    nb_yes = 0
    nb_mod = 0
    with Solver(bootstrap_with=fla.clauses) as l:
        print("at the beginning, fla is", l.solve(assumptions=[]))
        print("=> ADDING FEATURES FROM THE .CONFIG FILE [BOOL&TRISTATE]")
        for k, v in variables.items():
            if v in config_dict:
                if (v.endswith("_MODULE") or "CHOICE_" in v):  
                    continue # ignore
                kconfig_type = search_kconfig_type(v, df_options)
                if (kconfig_type == 'TRISTATE'):
                    # we have to find the other "module" variable that emulates the "m" value 
                    kmodule = search_kmodule(v)
                    # YES
                    if config_dict[v] == 'y':
                        print("+ Adding {} ({})".format(v, k))
                        print("+ Adding {} ({})".format(v + "_MODULE", -kmodule))
                        assumptions.extend([k, -kmodule])
                        nb_yes += 1
                    # MODULE
                    elif config_dict[v] == 'm':
                        print("+ Adding {} ({})".format(v, -k))
                        print("+ Adding {} ({})".format(v + "_MODULE", kmodule))
                        assumptions.extend([-k, kmodule])
                        nb_mod += 1
                    # SAT Testing
                    if not l.solve(assumptions=assumptions):
                        print("== UNSAT")
                        core = list(map((lambda x : '~' + variables[abs(x)] if (x < 0) else variables[x]),
                                        l.get_core()))
                        print(core)
                        return {"sat" : False,
                                "last_clause" : v,
                                "in" : True,
                                "core" : core}
        print("=> ADDING FEATURES THAT ARE NOT IN THE .CONFIG FILE [BOOL&TRISTATE]")
        for f in df_options.option:
            # Feature in the CSV and in the DIMACS but not in the config
            if f not in config and f in variables.values():
                k = get_var(f)
                kconfig_type = search_kconfig_type(f, df_options)
                if (kconfig_type == 'TRISTATE'):
                    kmodule = search_kmodule(f)
                    # NO
                    print("+ Adding {} ({})".format(f, -k))
                    print("+ Adding {} ({})".format(f + "_MODULE", -kmodule))
                    assumptions.extend([-k, -kmodule])
                    # SAT Testing
                    if not l.solve(assumptions=assumptions):
                        print("== UNSAT")
                        core = list(map((lambda x : '~' + variables[abs(x)] if (x < 0) else variables[x]),
                                        l.get_core()))
                        print(core)
                        return {"sat" : False,
                            "last_clause" : f,
                            "in" : True,
                            "core" : core}
        print("== SAT")
        return {"sat" : False,
                "last_clause" : None,
                "in" : True,
                "core" : None}

In [92]:
check_tristate(config)

at the beginning, fla is True
=> ADDING FEATURES FROM THE .CONFIG FILE [BOOL&TRISTATE]
+ Adding NLS_MAC_GAELIC (54865)
+ Adding NLS_MAC_GAELIC_MODULE (-54864)
+ Adding VIDEO_STI_HDMI_CEC (25376)
+ Adding VIDEO_STI_HDMI_CEC_MODULE (-25377)
+ Adding HID_STEELSERIES (54988)
+ Adding HID_STEELSERIES_MODULE (-54987)
+ Adding MFD_TPS65086 (13261)
+ Adding MFD_TPS65086_MODULE (-13262)
+ Adding SERIAL_ARC (27267)
+ Adding SERIAL_ARC_MODULE (-27266)
+ Adding MEDIA_TUNER_TEA5761 (-45839)
+ Adding MEDIA_TUNER_TEA5761_MODULE (45838)
+ Adding DMA_VIRTUAL_CHANNELS (37743)
+ Adding DMA_VIRTUAL_CHANNELS_MODULE (-37742)
+ Adding CRYPTO_CRYPTD (21863)
+ Adding CRYPTO_CRYPTD_MODULE (-21864)
+ Adding SOC_CAMERA_OV9640 (-19022)
+ Adding SOC_CAMERA_OV9640_MODULE (19021)
+ Adding PDA_POWER (4307)
+ Adding PDA_POWER_MODULE (-4306)
+ Adding CRYPTO_AUTHENC (21895)
+ Adding CRYPTO_AUTHENC_MODULE (-21896)
+ Adding TOUCHSCREEN_ST1232 (44828)
+ Adding TOUCHSCREEN_ST1232_MODULE (-44827)
+ Adding MOUSE_GPIO (32172)
+

+ Adding REGMAP_SPI (19550)
+ Adding REGMAP_SPI_MODULE (-19553)
+ Adding MTD_NAND_GPMI_NAND (-71815)
+ Adding MTD_NAND_GPMI_NAND_MODULE (71814)
+ Adding LIBNVDIMM (2640)
+ Adding LIBNVDIMM_MODULE (-2642)
+ Adding RTC_DRV_M41T80 (-63335)
+ Adding RTC_DRV_M41T80_MODULE (63334)
+ Adding PARPORT (-1547)
+ Adding PARPORT_MODULE (1548)
+ Adding BINFMT_MISC (1484)
+ Adding BINFMT_MISC_MODULE (-1483)
+ Adding RTC_DRV_PCF2123 (-53736)
+ Adding RTC_DRV_PCF2123_MODULE (53735)
+ Adding FMC_WRITE_EEPROM (70911)
+ Adding FMC_WRITE_EEPROM_MODULE (-70910)
+ Adding BATTERY_SBS (9402)
+ Adding BATTERY_SBS_MODULE (-9401)
+ Adding CRYPTO_CHACHA20 (2956)
+ Adding CRYPTO_CHACHA20_MODULE (-2955)
+ Adding EDD (-11308)
+ Adding EDD_MODULE (11307)
+ Adding MSPRO_BLOCK (73095)
+ Adding MSPRO_BLOCK_MODULE (-73094)
+ Adding FB_CFB_FILLRECT (72597)
+ Adding FB_CFB_FILLRECT_MODULE (-72596)
+ Adding SERIAL_DEV_BUS (-18278)
+ Adding SERIAL_DEV_BUS_MODULE (18277)
+ Adding MFD_STM32_TIMERS (12476)
+ Adding MFD_STM32_TIM

+ Adding INTERVAL_TREE_TEST (66198)
+ Adding INTERVAL_TREE_TEST_MODULE (-66197)
+ Adding CRYPTO_GF128MUL (51843)
+ Adding CRYPTO_GF128MUL_MODULE (-51842)
+ Adding CRYPTO_LZO (-5979)
+ Adding CRYPTO_LZO_MODULE (5978)
+ Adding HID_SUNPLUS (-51642)
+ Adding HID_SUNPLUS_MODULE (51641)
+ Adding NLS_ISO8859_4 (16792)
+ Adding NLS_ISO8859_4_MODULE (-16791)
+ Adding VIDEOBUF2_DMA_SG (3507)
+ Adding VIDEOBUF2_DMA_SG_MODULE (-3508)
+ Adding RTC_DRV_PCF8563 (-71472)
+ Adding RTC_DRV_PCF8563_MODULE (71471)
+ Adding TPS6105X (10716)
+ Adding TPS6105X_MODULE (-10717)
+ Adding NLS_CODEPAGE_437 (55852)
+ Adding NLS_CODEPAGE_437_MODULE (-55851)
+ Adding FSCACHE (-7318)
+ Adding FSCACHE_MODULE (7319)
+ Adding SPI_STM32 (-49156)
+ Adding SPI_STM32_MODULE (49155)
+ Adding REED_SOLOMON (51164)
+ Adding REED_SOLOMON_MODULE (-51163)
+ Adding CRC_ITU_T (15035)
+ Adding CRC_ITU_T_MODULE (-15034)
+ Adding VIDEO_ADV7343 (-24329)
+ Adding VIDEO_ADV7343_MODULE (24328)
+ Adding PCWATCHDOG (-32027)
+ Adding PCWATCHD

+ Adding F2FS_FS (-7137)
+ Adding F2FS_FS_MODULE (7136)
+ Adding BLK_DEV_DM (1093)
+ Adding BLK_DEV_DM_MODULE (-1092)
+ Adding SOC_CAMERA_OV9740 (-26623)
+ Adding SOC_CAMERA_OV9740_MODULE (26622)
+ Adding TOUCHSCREEN_ELAN (54452)
+ Adding TOUCHSCREEN_ELAN_MODULE (-54451)
+ Adding GPIO_LP873X (-21300)
+ Adding GPIO_LP873X_MODULE (21299)
+ Adding RTC_DRV_FM3130 (50494)
+ Adding RTC_DRV_FM3130_MODULE (-50493)
+ Adding IT8712F_WDT (-45148)
+ Adding IT8712F_WDT_MODULE (45147)
+ Adding I2C_ALGOPCF (43103)
+ Adding I2C_ALGOPCF_MODULE (-43102)
+ Adding CRYPTO_DEV_PADLOCK (46595)
+ Adding CRYPTO_DEV_PADLOCK_MODULE (-46594)
+ Adding CRYPTO_TGR192 (-33651)
+ Adding CRYPTO_TGR192_MODULE (33650)
+ Adding NLS_UTF8 (56576)
+ Adding NLS_UTF8_MODULE (-56575)
+ Adding FB_OPENCORES (-60622)
+ Adding FB_OPENCORES_MODULE (60621)
+ Adding I2C_AXXIA (74535)
+ Adding I2C_AXXIA_MODULE (-74534)
+ Adding TOUCHSCREEN_CYTTSP_CORE (-44728)
+ Adding TOUCHSCREEN_CYTTSP_CORE_MODULE (44727)
+ Adding CRYPTO_JITTERENTROP

+ Adding TCG_TIS_I2C_ATMEL (-42669)
+ Adding TCG_TIS_I2C_ATMEL_MODULE (42668)
+ Adding NLS_MAC_CENTEURO (38493)
+ Adding NLS_MAC_CENTEURO_MODULE (-38492)
+ Adding HID_ASUS (38692)
+ Adding HID_ASUS_MODULE (-38691)
+ Adding VIDEO_ADV7170 (39314)
+ Adding VIDEO_ADV7170_MODULE (-39313)
+ Adding MOUSE_CYAPA (-15046)
+ Adding MOUSE_CYAPA_MODULE (15043)
+ Adding FW_CFG_SYSFS (-22533)
+ Adding FW_CFG_SYSFS_MODULE (22532)
+ Adding MMC_WBSD (-50962)
+ Adding MMC_WBSD_MODULE (50961)
+ Adding MEDIA_TUNER_TEA5767 (29049)
+ Adding MEDIA_TUNER_TEA5767_MODULE (-29048)
+ Adding X86_CPUID (-15071)
+ Adding X86_CPUID_MODULE (15070)
+ Adding REGULATOR_TPS6507X (-7780)
+ Adding REGULATOR_TPS6507X_MODULE (7779)
+ Adding SENSORS_LIS3LV02D (32159)
+ Adding SENSORS_LIS3LV02D_MODULE (-32160)
+ Adding FB_TFT_ST7789V (-52046)
+ Adding FB_TFT_ST7789V_MODULE (52045)
+ Adding CONFIGFS_FS (9161)
+ Adding CONFIGFS_FS_MODULE (-9163)
+ Adding FB_TFT_AGM1264K_FL (-15636)
+ Adding FB_TFT_AGM1264K_FL_MODULE (15635)
+ Addi

...

+ Adding BLK_DEV_DELKIN (-19057)
+ Adding BLK_DEV_DELKIN_MODULE (-19056)
+ Adding VETH (-10436)
+ Adding VETH_MODULE (-10435)
+ Adding PCMCIA_FMVJ18X (-23585)
+ Adding PCMCIA_FMVJ18X_MODULE (-23584)
+ Adding SND_SOC_LOWLAND (-36456)
+ Adding SND_SOC_LOWLAND_MODULE (-36457)
+ Adding EDAC_PPC4XX (-9725)
+ Adding EDAC_PPC4XX_MODULE (-9724)
+ Adding COMEDI_8255_SA (-35152)
+ Adding COMEDI_8255_SA_MODULE (-35153)
+ Adding BERLIN2_ADC (-71217)
+ Adding BERLIN2_ADC_MODULE (-71216)
+ Adding IP_SET_HASH_MAC (-46217)
+ Adding IP_SET_HASH_MAC_MODULE (-46216)
+ Adding IBM_BSR (-19349)
+ Adding IBM_BSR_MODULE (-19348)
+ Adding SENSORS_THMC50 (-15695)
+ Adding SENSORS_THMC50_MODULE (-15694)
+ Adding NFC_FDP_I2C (-51711)
+ Adding NFC_FDP_I2C_MODULE (-51710)
+ Adding BLK_DEV_IDECD (-64135)
+ Adding BLK_DEV_IDECD_MODULE (-64136)
+ Adding VIRTIO_NET (-72055)
+ Adding VIRTIO_NET_MODULE (-72054)
+ Adding SND_SOC_WM5100 (-53920)
+ Adding SND_SOC_WM5100_MODULE (-53919)
+ Adding EDAC_AMD8131 (-42040)
+ Addin

+ Adding ZFCP (-76607)
+ Adding ZFCP_MODULE (-76606)
+ Adding FB_PXA (-16102)
+ Adding FB_PXA_MODULE (-16101)
+ Adding EDAC_CPC925 (-66100)
+ Adding EDAC_CPC925_MODULE (-66099)
+ Adding NFC_PN544_MEI (-26478)
+ Adding NFC_PN544_MEI_MODULE (-26477)
+ Adding EDAC_TILE (-33653)
+ Adding EDAC_TILE_MODULE (-33652)
+ Adding NTB_TOOL (-70886)
+ Adding NTB_TOOL_MODULE (-70885)
+ Adding NET_IPIP (-17535)
+ Adding NET_IPIP_MODULE (-17534)
+ Adding IP_SET_HASH_NETPORT (-74966)
+ Adding IP_SET_HASH_NETPORT_MODULE (-74965)
+ Adding NFC_PN533 (-31897)
+ Adding NFC_PN533_MODULE (-31896)
+ Adding SCSI_PMCRAID (-24077)
+ Adding SCSI_PMCRAID_MODULE (-24076)
+ Adding IPMI_DEVICE_INTERFACE (-45357)
+ Adding IPMI_DEVICE_INTERFACE_MODULE (-45356)
+ Adding W1_SLAVE_DS2438 (-18942)
+ Adding W1_SLAVE_DS2438_MODULE (-18941)
+ Adding EDAC_HIGHBANK_MC (-15715)
+ Adding EDAC_HIGHBANK_MC_MODULE (-15714)
+ Adding RTLLIB (-4544)
+ Adding RTLLIB_MODULE (-4545)
+ Adding NTB_PERF (-71129)
+ Adding NTB_PERF_MODULE (-7112

+ Adding IR_JVC_DECODER (-51467)
+ Adding IR_JVC_DECODER_MODULE (-51468)
+ Adding SND_SB16 (-6103)
+ Adding SND_SB16_MODULE (-6104)
+ Adding TOUCHSCREEN_EGALAX (-5265)
+ Adding TOUCHSCREEN_EGALAX_MODULE (-5264)
+ Adding ARCNET_RIM_I (-19880)
+ Adding ARCNET_RIM_I_MODULE (-19879)
+ Adding DVB_EC100 (-59590)
+ Adding DVB_EC100_MODULE (-59589)
+ Adding DRM_DW_HDMI_I2S_AUDIO (-22005)
+ Adding DRM_DW_HDMI_I2S_AUDIO_MODULE (-22006)
+ Adding VMWARE_VMCI_VSOCKETS (-15960)
+ Adding VMWARE_VMCI_VSOCKETS_MODULE (-15959)
+ Adding HNS_MDIO (-16163)
+ Adding HNS_MDIO_MODULE (-16164)
+ Adding SERIAL_8250_HP300 (-9984)
+ Adding SERIAL_8250_HP300_MODULE (-9983)
+ Adding SPI_BCM53XX (-26050)
+ Adding SPI_BCM53XX_MODULE (-26049)
+ Adding IR_SONY_DECODER (-43412)
+ Adding IR_SONY_DECODER_MODULE (-43411)
+ Adding SND_SBAWE (-2754)
+ Adding SND_SBAWE_MODULE (-2756)
+ Adding PWM_AB8500 (-17597)
+ Adding PWM_AB8500_MODULE (-17596)
+ Adding ARCNET_COM20020 (-3031)
+ Adding ARCNET_COM20020_MODULE (-3030)
+ Addi

+ Adding TEST_LIST_SORT (-71213)
+ Adding TEST_LIST_SORT_MODULE (-71212)
+ Adding PCMCIA_QLOGIC (-73649)
+ Adding PCMCIA_QLOGIC_MODULE (-73648)
+ Adding DVB_RTL2832 (-19602)
+ Adding DVB_RTL2832_MODULE (-19603)
+ Adding SOUND_SB (-57003)
+ Adding SOUND_SB_MODULE (-57002)
+ Adding SND_SOC_TLV320AIC3X (-57357)
+ Adding SND_SOC_TLV320AIC3X_MODULE (-57356)
+ Adding TOSHIBA_BT_RFKILL (-50107)
+ Adding TOSHIBA_BT_RFKILL_MODULE (-50106)
+ Adding TEST_SORT (-50946)
+ Adding TEST_SORT_MODULE (-50945)
+ Adding PCMCIA_SYM53C500 (-67749)
+ Adding PCMCIA_SYM53C500_MODULE (-67748)
+ Adding DVB_RTL2832_SDR (-39460)
+ Adding DVB_RTL2832_SDR_MODULE (-39459)
+ Adding SOUND_YM3812 (-27975)
+ Adding SOUND_YM3812_MODULE (-27974)
+ Adding R8822BE (-44667)
+ Adding R8822BE_MODULE (-44666)
+ Adding SND_AM33XX_SOC_EVM (-28925)
+ Adding SND_AM33XX_SOC_EVM_MODULE (-28924)
+ Adding TOSHIBA_HAPS (-31974)
+ Adding TOSHIBA_HAPS_MODULE (-31973)
+ Adding SOUND_UART6850 (-18907)
+ Adding SOUND_UART6850_MODULE (-18906)


{'sat': False, 'last_clause': None, 'in': True, 'core': None}

In [94]:
def check_bool(config_dict):
    fla = CNF(DIMACS_FLA)
    # we read once and for all
    df_options = pd.read_csv(CSV)
    assumptions = []
    nb_yes = 0
    nb_mod = 0
    with Solver(bootstrap_with=fla.clauses) as l:
        print("at the beginning, fla is", l.solve(assumptions=[]))
        print("=> ADDING FEATURES FROM THE .CONFIG FILE [BOOL&TRISTATE]")
        for k, v in variables.items():
            if v in config_dict:
                if (v.endswith("_MODULE") or "CHOICE_" in v):  
                    continue # ignore
                kconfig_type = search_kconfig_type(v, df_options)
                if (kconfig_type == 'BOOL'):
                    # we have to find the other "module" variable that emulates the "m" value 
                    kmodule = search_kmodule(v)
                    print("+ Adding {} ({})".format(v, k))
                    assumptions.extend([k])
                    # SAT Testing
                    if not l.solve(assumptions=assumptions):
                        print("== UNSAT")
                        core = list(map((lambda x : '~' + variables[abs(x)] if (x < 0) else variables[x]),
                                        l.get_core()))
                        print(core)
                        return {"sat" : False,
                                "last_clause" : v,
                                "in" : True,
                                "core" : core}
        print("=> ADDING FEATURES THAT ARE NOT IN THE .CONFIG FILE [BOOL&TRISTATE]")
        for f in df_options.option:
            # Feature in the CSV and in the DIMACS but not in the config
            if f not in config and f in variables.values():
                k = get_var(f)
                kconfig_type = search_kconfig_type(f, df_options)
                if (kconfig_type == 'BOOL'):
                    kmodule = search_kmodule(f)
                    # NO
                    print("+ Adding {} ({})".format(f, -k))
                    assumptions.extend([-k])
                    # SAT Testing
                    if not l.solve(assumptions=assumptions):
                        print("== UNSAT")
                        core = list(map((lambda x : '~' + variables[abs(x)] if (x < 0) else variables[x]),
                                        l.get_core()))
                        print(core)
                        return {"sat" : False,
                            "last_clause" : f,
                            "in" : True,
                            "core" : core}
        print("== SAT")
        return {"sat" : False,
                "last_clause" : None,
                "in" : True,
                "core" : None}


In [95]:
check_bool(config)

at the beginning, fla is True
=> ADDING FEATURES FROM THE .CONFIG FILE [BOOL&TRISTATE]
+ Adding X86_CMPXCHG64 (23722)
+ Adding SRAM (15364)
+ Adding HAVE_DMA_CONTIGUOUS (10160)
+ Adding OMAP5_THERMAL (4904)
+ Adding HAVE_RCU_TABLE_FREE (10031)
+ Adding ISA_BUS (25820)
+ Adding CGROUP_RDMA (53950)
+ Adding IRQ_UNIPHIER_AIDET (5941)
+ Adding KALLSYMS_BASE_RELATIVE (44599)
+ Adding TWL6040_CORE (923)
+ Adding GENERIC_FIND_FIRST_BIT (17795)
+ Adding ARCH_HAS_ELF_RANDOMIZE (29368)
+ Adding WATCHDOG_NOWAYOUT (15758)
+ Adding WATCHDOG (976)
+ Adding ARCH_HAS_SG_CHAIN (53949)
+ Adding H8300_TPU (57071)
+ Adding MFD_TPS80031 (13110)
+ Adding ARCH_HAS_STRICT_KERNEL_RWX (55708)
+ Adding BLK_CGROUP (26621)
+ Adding KSM (27152)
+ Adding SPI_MASTER (231)
+ Adding CGROUP_PERF (60778)
+ Adding GLOB (25452)
+ Adding DEBUG_STACKOVERFLOW (31834)
+ Adding MTRR_SANITIZER (60272)
+ Adding GENERIC_HWEIGHT (53270)
+ Adding CRC32_BIT (59357)
+ Adding LOCKDEP_SUPPORT (7124)
+ Adding FB_DEFERRED_IO (3241)
+ Addi

+ Adding QCOM_EBI2 (51251)
+ Adding HAVE_PERF_REGS (406)
+ Adding ZSMALLOC_STAT (45880)
+ Adding HAVE_KERNEL_BZIP2 (405)
+ Adding MFD_MAX8925 (146)
+ Adding LOGO (26615)
+ Adding BCH_CONST_PARAMS (75144)
+ Adding ARCH_SUPPORTS_ATOMIC_RMW (27663)
+ Adding MD (1094)
+ Adding BCM_KONA_WDT_DEBUG (57393)
+ Adding PREEMPT_COUNT (24971)
+ Adding FSNOTIFY (20978)
+ Adding REED_SOLOMON_DEC8 (74007)
+ Adding F2FS_FS_ENCRYPTION (31947)
+ Adding CRYPTO_HW (2026)
+ Adding PROVE_LOCKING (7165)
+ Adding SERIAL_IMX_CONSOLE (17539)
+ Adding GENERIC_IRQ_CHIP (6670)
+ Adding EFI_PARTITION (6024)
+ Adding HZ_1000 (19635)
+ Adding DEBUG_LOCKDEP (48886)
+ Adding HAVE_SETUP_PER_CPU_AREA (36580)
+ Adding RMI4_2D_SENSOR (43773)
+ Adding VIDEO_ADV_DEBUG (66169)
+ Adding KGDB (3600)
+ Adding NTFS_RW (40039)
+ Adding I2C_SLAVE (7092)
+ Adding PERF_USE_VMALLOC (12653)
+ Adding I2C_DEBUG_CORE (73053)
+ Adding BPF_SYSCALL (6211)
+ Adding HDMI (35051)
+ Adding BUILDTIME_EXTABLE_SORT (65524)
+ Adding CLKSRC_I8253 (429

+ Adding PREEMPT (630)
+ Adding FLATMEM (50204)
+ Adding COMPAT_32 (41269)
+ Adding TTY (31)
+ Adding HAVE_MIXED_BREAKPOINTS_REGS (71470)
+ Adding XZ_DEC_POWERPC (47375)
+ Adding HAS_DMA (431)
+ Adding IRQ_WORK (57415)
+ Adding MOUSE_ELAN_I2C_SMBUS (58938)
+ Adding HAVE_PERF_EVENTS_NMI (46332)
+ Adding STRICT_MODULE_RWX (55309)
+ Adding BLK_DEV_BSGLIB (27499)
+ Adding ARCH_HAS_REFCOUNT (24391)
+ Adding BLK_DEBUG_FS (75142)
+ Adding ARCH_ENABLE_SPLIT_PMD_PTLOCK (46333)
+ Adding NAMESPACES (4406)
+ Adding PCI_LOCKLESS_CONFIG (67028)
+ Adding RANDOMIZE_BASE (41938)
+ Adding CC_STACKPROTECTOR (42944)
+ Adding SHMEM (28606)
+ Adding ARCH_HAS_CPU_RELAX (60703)
+ Adding ARCH_HAS_CACHE_LINE_SIZE (19000)
+ Adding ISA_BUS_API (11103)
+ Adding LEDS_PCA9532_GPIO (56434)
+ Adding DEBUG_WQ_FORCE_RR_CPU (38429)
+ Adding HAVE_DEBUG_STACKOVERFLOW (7144)
+ Adding MULTIUSER (5638)
+ Adding ADVISE_SYSCALLS (72991)
+ Adding MOUSE_PS2_SYNAPTICS_SMBUS (3885)
+ Adding VMSPLIT_3G (9593)
+ Adding PREEMPT_RCU (2

+ Adding BCMA_DEBUG (-70134)
+ Adding B43_LEDS (-50680)
+ Adding HISAX_GAZEL (-61381)
+ Adding FW_CFG_SYSFS_CMDLINE (-38442)
+ Adding NET_FOU_IP_TUNNELS (-17532)
+ Adding B43_HWRNG (-43816)
+ Adding HISAX_HFC_PCI (-73220)
+ Adding QCOM_SCM_32 (-18825)
+ Adding SCHED_HRTICK (-42408)
+ Adding MFD_AS3711 (-1973)
+ Adding B43_DEBUG (-51960)
+ Adding QCOM_SCM_64 (-65033)
+ Adding HISAX_W6692 (-74933)
+ Adding MFD_AAT2870_CORE (-31753)
+ Adding BCM47XX_SPROM (-9087)
+ Adding KEXEC_FILE (-42379)
+ Adding HISAX_HFC_SX (-61877)
+ Adding B43LEGACY_PCI_AUTOSELECT (-30820)
+ Adding GOOGLE_FIRMWARE (-4501)
+ Adding HAVE_LIVEPATCH (-60673)
+ Adding KEXEC_VERIFY_SIG (-15130)
+ Adding HISAX_ENTERNOW_PCI (-67173)
+ Adding PWM_CRC (-31059)
+ Adding B43LEGACY_PCICORE_AUTOSELECT (-44572)
+ Adding SND_SB16_CSP (-17051)
+ Adding ATM_DRIVERS (-2707)
+ Adding PINCTRL_PFC_R8A7795 (-26817)
+ Adding USB_INVENTRA_DMA (-30161)
+ Adding USB_TI_CPPI_DMA (-40751)
+ Adding SERIAL_8250_EXTENDED (-6328)
+ Adding PINCTRL

+ Adding CIFS_SMB311 (-47289)
+ Adding MFD_ASIC3 (-6804)
+ Adding CIFS_FSCACHE (-29411)
+ Adding PARAVIRT_DEBUG (-41940)
+ Adding USB_DWC2_DEBUG (-14918)
+ Adding IBM_EMAC_NO_FLOW_CTRL (-59709)
+ Adding PINCTRL_PFC_SH7757 (-8858)
+ Adding DVB_B2C2_FLEXCOP_DEBUG (-53602)
+ Adding IBM_EMAC_MAL_CLR_ICINTSTAT (-72936)
+ Adding TEGRA_HSP_MBOX (-52793)
+ Adding IIO_ADIS_LIB_BUFFER (-28828)
+ Adding IBM_EMAC_MAL_COMMON_ERR (-37628)
+ Adding CAPI_TRACE (-63264)
+ Adding IIO_BUFFER (-3421)
+ Adding PINCTRL_PFC_SH7785 (-75854)
+ Adding ARCH_TEGRA_186_SOC (-29006)
+ Adding SND_BT87X_OVERCLOCK (-19887)
+ Adding ISDN_CAPI_MIDDLEWARE (-72435)
+ Adding PINCTRL_PFC_SH7786 (-90)
+ Adding HAVE_KVM_IRQ_BYPASS (-50730)
+ Adding NET_VENDOR_INTEL (-603)
+ Adding HOTPLUG_CPU (-4212)
+ Adding BOOTPARAM_HOTPLUG_CPU0 (-16137)
+ Adding NET_L3_MASTER_DEV (-39362)
+ Adding ISDN_CAPI_CAPIDRV_VERBOSE (-39565)
+ Adding PINCTRL_PFC_SHX3 (-70937)
+ Adding DEBUG_HOTPLUG_CPU0 (-49864)
+ Adding CAPI_AVM (-7195)
+ Adding D

...

+ Adding INFINIBAND_IPOIB_DEBUG_DATA (-45265)
+ Adding PINCTRL_DOVE (-15863)
+ Adding GPIO_MSIC (-29821)
+ Adding SND_DMA_SGBUF (-8008)
+ Adding SUN4I_TIMER (-34337)
+ Adding ROMFS_BACKED_BY_BLOCK (-6318)
+ Adding ADB_CUDA (-48007)
+ Adding NET_VENDOR_ROCKER (-26058)
+ Adding USB_SL811_HCD_ISO (-48830)
+ Adding RCU_TRACE (-47461)
+ Adding ARM_S3C2416_CPUFREQ (-12695)
+ Adding ADB_PMU_LED (-22447)
+ Adding SCSI_NCR53C8XX_NO_DISCONNECT (-47652)
+ Adding TRACE_CLOCK (-60268)
+ Adding ADB_PMU_LED_DISK (-45828)
+ Adding NET_VENDOR_SAMSUNG (-14481)
+ Adding SYSTEM_EXTRA_CERTIFICATE (-18263)
+ Adding ARM_S3C2416_CPUFREQ_VCORESCALE (-73338)
+ Adding ARM_S3C2440_CPUFREQ (-67364)
+ Adding PMAC_SMU (-15726)
+ Adding DEBUG_BLOCK_EXT_DEVT (-73089)
+ Adding NET_VENDOR_SEEQ (-26041)
+ Adding DSCC4_PCISYNC (-10057)
+ Adding CPU_HOTPLUG_STATE_CONTROL (-29990)
+ Adding PMAC_MEDIABAY (-30351)
+ Adding DSCC4_PCI_RST (-6579)
+ Adding NET_CLS_ACT (-101)
+ Adding BTREE (-968)
+ Adding PMAC_BACKLIGHT (-36168)

+ Adding GPIO_TPS68470 (-66539)
+ Adding USB_CONFIGFS_F_LB_SS (-26693)
+ Adding CMA_DEBUGFS (-74130)
+ Adding CB710_DEBUG_ASSUMPTIONS (-53440)
+ Adding SENSORS_LTC2978_REGULATOR (-36679)
+ Adding FAULT_INJECTION_STACKTRACE_FILTER (-29403)
+ Adding NET_EGRESS (-16511)
+ Adding MFD_TPS68470 (-1269)
+ Adding USB_CONFIGFS_F_FS (-21413)
+ Adding LATENCYTOP (-7126)
+ Adding CARL9170_LEDS (-33760)
+ Adding USB_CONFIGFS_F_UAC1 (-4251)
+ Adding CLS_U32_PERF (-60590)
+ Adding OMAP2_VRFB (-9652)
+ Adding PXA_DMA (-15997)
+ Adding NOP_TRACER (-22870)
+ Adding CARL9170_DEBUGFS (-17787)
+ Adding USB_CONFIGFS_F_UAC1_LEGACY (-9821)
+ Adding MEM_SOFT_DIRTY (-18984)
+ Adding STE_DMA40 (-107)
+ Adding HAVE_FTRACE_NMI_ENTER (-7898)
+ Adding CLS_U32_MARK (-70235)
+ Adding CARL9170_WPC (-39383)
+ Adding USB_CONFIGFS_F_UAC2 (-1465)
+ Adding ZSWAP (-4609)
+ Adding TRACER_MAX_TRACE (-46099)
+ Adding CARL9170_HWRNG (-49905)
+ Adding USB_CONFIGFS_F_MIDI (-3559)
+ Adding FTRACE_NMI_ENTER (-7899)
+ Adding USB_CONF

+ Adding FTRACE_SYSCALLS (-7128)
+ Adding W1_CON (-61788)
+ Adding HARDLOCKUP_CHECK_TIMESTAMP (-43881)
+ Adding TRACE_BRANCH_PROFILING (-16495)
+ Adding HAVE_ACPI_APEI (-30804)
+ Adding HAVE_ACPI_APEI_NMI (-53293)
+ Adding BRANCH_PROFILE_NONE (-35493)
+ Adding PINCTRL_SAMSUNG (-8240)
+ Adding MUTEX_SPIN_ON_OWNER (-36778)
+ Adding ATH10K_DEBUGFS (-10618)
+ Adding MIGRATE_VMA_HELPER (-7349)
+ Adding RWSEM_SPIN_ON_OWNER (-27664)
+ Adding CAIF_DEBUG (-2623)
+ Adding ATH10K_TRACING (-65454)
+ Adding INTEL_TH_DEBUG (-43882)
+ Adding HMM (-7348)
+ Adding LOCK_SPIN_ON_OWNER (-36777)
+ Adding ATH10K_DFS_CERTIFIED (-28093)
+ Adding GOLDFISH_TTY_EARLY_CONSOLE (-62176)
+ Adding HMM_MIRROR (-20369)
+ Adding QUEUED_SPINLOCKS (-63955)
+ Adding DEVICE_PRIVATE (-46978)
+ Adding QUEUED_RWLOCKS (-18388)
+ Adding WCN36XX_DEBUGFS (-47998)
+ Adding DA_TTY (-29693)
+ Adding DEVICE_PUBLIC (-46977)
+ Adding SMSC911X_ARCH_HOOKS (-11044)
+ Adding FANOTIFY_ACCESS_PERMISSIONS (-47811)
+ Adding CC_OPTIMIZE_FOR_SIZE

+ Adding OXNAS_RPS_TIMER (-34369)
+ Adding IDE_GD_ATAPI (-64139)
+ Adding SECURITY_PATH (-35036)
+ Adding SYS_SUPPORTS_SH_CMT (-10742)
+ Adding TUN_VNET_CROSS_LE (-54707)
+ Adding NET_VENDOR_FUJITSU (-22163)
+ Adding PINCTRL_PFC_R8A7740 (-38834)
+ Adding MTK_TIMER (-34367)
+ Adding NET_CLS_IND (-20485)
+ Adding SYS_SUPPORTS_SH_MTU2 (-50333)
+ Adding HAVE_CONTEXT_TRACKING (-18368)
+ Adding NET_VENDOR_HISILICON (-15813)
+ Adding DCB (-4343)
+ Adding PINCTRL_PFC_R8A7743 (-30953)
+ Adding SYS_SUPPORTS_SH_TMU (-20488)
+ Adding BLK_DEV_IDECD_VERBOSE_ERRORS (-73385)
+ Adding SYS_SUPPORTS_EM_STI (-55547)
+ Adding INTEL_TXT (-7475)
+ Adding HISAX_FRITZPCI (-30793)
+ Adding BCM47XX_NVRAM (-9088)
+ Adding B43_BUSES_BCMA (-14668)
+ Adding NFSD_PNFS (-65780)
+ Adding HISAX_AVM_A1_PCMCIA (-42409)
+ Adding SSB_SILENT (-6826)
+ Adding B43_BUSES_SSB (-25778)
+ Adding W1_SLAVE_DS2408_READBACK (-36081)
+ Adding NFSD_BLOCKLAYOUT (-31643)
+ Adding HISAX_ELSA (-28511)
+ Adding B43_PCI_AUTOSELECT (-4840)
+ A

{'sat': False, 'last_clause': None, 'in': True, 'core': None}

In [96]:
def checker(n):
    stream = open("examples/extractor_out.csv", "w")
    stream.write("name,sat,nb_yes,nb_mod,last_clause,in,core\n")
    for i in range(n):
        for flag in CONF_FLAG:
            os.system("make {} -C {}".format(flag, CONFIG_DIR))
            config_dict = get_dict(CONFIG)
            tristate_nb = tristate_repartition(config_dict)
            tri_checker = check_tristate(config_dict)
            bool_checker = check_bool(config_dict)
            res_checker = check_both(config_dict)
            name = "{}_{}".format(flag, i)
            core = None
            if res_checker["core"] is not None:
                core = "{}".format(res_checker["core"]).replace(',', ' ')
            
            stream.write("{},{},{},{},{},{},{}\n".format(name, res_checker["sat"], tristate_nb["yes"],
                                                      tristate_nb["mod"], res_checker["last_clause"],
                                                      res_checker["in"], core))
    stream.close()